<a href="https://colab.research.google.com/github/vixbi/actprobl/blob/main/Shumakova_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix
from collections import Counter
from nltk.tokenize import sent_tokenize

## Задание 1

In [ ]:
dvach = open('2ch_corpus.txt').read()

In [ ]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

# в конце отложила первые 100 предложений
sentences_dvach =  [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dvach[:5000000])][100:]
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

unigrams_dvach = Counter()
bigram_dvach = Counter()
trigram_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigram_dvach.update(ngrammer(sentence, n=2))
    trigram_dvach.update(ngrammer(sentence, n=3))

In [ ]:
matrix_dvach = lil_matrix((len(bigram_dvach), len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}
id2bigrams_dvach = list(bigram_dvach)
word2idbigrams_dvach = {word:i for i, word in enumerate(id2bigrams_dvach)}

for ngram in trigram_dvach:
    word_bigram, word_unigram = ngram.rsplit(" ", 1)
    matrix_dvach[word2idbigrams_dvach[word_bigram], word2id_dvach[word_unigram]] =  (trigram_dvach[ngram]/
                                                                     bigram_dvach[word_bigram])
matrix_dvach = csc_matrix(matrix_dvach)

In [ ]:
def generate(matrix, id2word, word2id, n=100, start='<start> <start>'):
    text = []
    current_idx = word2id[start]
    previous = ["<start>"]

    for i in range(n):
        chosen_index = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen_index])

        chosen = id2word[chosen_index]
        text.append(chosen)
        if chosen == "<end>":
            previous = ["<start>"]
            chosen = "<start>"

        previous.append(chosen)
        current_idx = word2id[" ".join(previous)]
        previous = previous[1:]

    res_text = []
    prev = ''
    for i in text:
        if prev != i:
            res_text.append(i)
        prev = i

    return ' '.join(res_text).replace('<end>', '\n')

In [ ]:
#генерация 1
print(generate(matrix_dvach, id2word_dvach, word2idbigrams_dvach))

3 
 два чая иншаллах 
 все в хламину отсутствие безумных проков которые были после марки передаёте этот пиздёж про нирвану и возвышенное существование остаётся пиздежом поскольку во-первых его никто не верит никто и не млекопитающее 
 вкратце пётр дегтярев 
 в шапке треда 
 я чего-то не вижу мусора трупов алкашей и вот совсем адская мухосрань 
 и тут не текстовые рпг наоборот ж я пытаюсь вам открыть глаза 
 а можно как-то найти что-то общее ничего усачев не знает кто сбил боинг и кто ее трахал 
 я всегда охуеваю вот с замедлением вдруг стало лучше но никто


In [ ]:
#генерация 2
print(generate(matrix_dvach, id2word_dvach, word2idbigrams_dvach))

ты еще и зарабатывает на долге 
 боко харам в афганистан где как раз пройдет 
 быстренько да для ачивки 
 ёбаный сиетл заходишь на видеохостинг тытруба в топе стима тоже не хотел их делать 
 что за хуйню ты слушаешь пудель зачем ты сидишь тут праведный такой 
 скорее всего еще меньше 
 каких там солей говоришь надо 
 а дробить на 10 метрах 
 походу ей пизда пришла схуяли она промахивается на 10 морская свинка 
 ну и фантазии про по морде мы уже придали тебя забвению 
 не бомби ты так делай 
 игишня сама говорила что


In [ ]:
#генерация 3
print(generate(matrix_dvach, id2word_dvach, word2idbigrams_dvach))

одно дело где я теперь вне закона 
 может кто решил проблему 
 чтоб не обосраться 
 только в библиотеках ультрахардкор молодость мытищи 359 
 но четкий числовой ответ тут дать невозможно 
 мань перед тем как найти хоть шмотки какие нибудь судебные процессы и прочая хуйня не слушай его больше не согласится на использование тех данных которые уже купили их у меня где-то так лоал 
 нераскрытый потенциал спроси хотела бы я без нее кун 
 вы соринки в наших пердях годные терапевты водятся 
 ты же должен работать обман с оформлением по разным причинам даже если нет количество


Рассчет перплексии:

In [ ]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def compute_joint_proba(text, word_probas):
    prob = 0
    tokens = normalize(text)
    for word in tokens:
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += np.log(2e-4)

    return prob, len(tokens)


def compute_join_proba_markov_assumption(text, word_counts, bigram_counts):
    prob = 0
    tokens = normalize(phrase)
    for ngram in ngrammer(['<start>'] + tokens + ['<end>']):
        word1, word2 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)

    return prob, len(tokens)